In [188]:
import pandas as pd

def clean(week_nums):
    dfs = []
    for week_num in week_nums:
        url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
        file_url = url.format(week_num)
        names = ['c_a', 'unit', 'scp', 'station', 'linename', 'division', 
                 'date', 'time', 'desc', 'entries', 'exits']
        df = pd.read_csv(file_url, names=names, parse_dates=[['date','time']], 
                         keep_date_col=True, skiprows=1)
        df['date'] = pd.to_datetime(df['date'])
        df['day_of_week'] = df['date_time'].dt.day_name()
        df = df[(~df['c_a'].str.contains('PTH') & 
                 ~df['desc'].str.contains('RECOVR') & 
                 df.time.astype(str).str.contains('00:00'))]
        df = df[['station', 'unit', 'c_a', 'scp', 'date_time', 'date', 'day_of_week', 'time', 
                'desc', 'entries', 'exits']]
        #delete duplicates -> could be done after dataframe creation instead of in this loop
        df.sort_values(['c_a', 'unit', 'scp', 'station', 'date_time'], inplace=True, ascending=False)
        df.drop_duplicates(subset=['c_a', 'unit', 'scp', 'station', 'date_time'], inplace=True)
        dfs.append(df)
        print(week_num)
    return pd.concat(dfs)

week_nums = [160903, 160910, 160917]

In [189]:
turnstiles_df = clean(week_nums)

160903
160910
160917


In [197]:
turnstiles_df.date


193138   2016-09-02
193137   2016-09-02
193136   2016-09-02
193135   2016-09-02
193134   2016-09-02
            ...    
4        2016-09-10
3        2016-09-10
2        2016-09-10
1        2016-09-10
0        2016-09-10
Name: date, Length: 532709, dtype: datetime64[ns]

In [173]:
turnstiles_block = (turnstiles_df
                        .groupby(["c_a", "unit", "scp", "station", "date_time", "exits"],as_index=False).entries.first())
turnstiles_block[["prev_datetime", "prev_entries", "prev_exits"]] = (turnstiles_block
                                                       .groupby(["c_a", "unit", "scp", "station"])["date_time", "entries", "exits"]
                                                       .apply(lambda grp: grp.shift(1)))
turnstiles_block.dropna(subset=["prev_datetime"], axis=0, inplace=True)
def get_counts_entry(row, max_counter):
    counter = row["entries"] - row["prev_entries"]
    if counter < 0:
        # Maybe counter is reversed?
        counter = -counter
    if counter > max_counter:
        # Maybe counter was reset to 0? 
        print(row["entries"], row["prev_entries"])
        counter = min(row["entries"], row["prev_entries"])
    if counter > max_counter:
        # Check it again to make sure we're not still giving a counter that's too big
        return 0
    return counter
def get_counts_exit(row, max_counter):
    counter = row["exits"] - row["prev_exits"]
    if counter < 0:
        # Maybe counter is reversed?
        counter = -counter
    if counter > max_counter:
        # Maybe counter was reset to 0? 
        print(row["exits"], row["prev_exits"])
        counter = min(row["exits"], row["prev_exits"])
    if counter > max_counter:
        # Check it again to make sure we're not still giving a counter that's too big
        return 0
    return counter

<ipython-input-173-7112b57ef14a>:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  turnstiles_block[["prev_datetime", "prev_entries", "prev_exits"]] = (turnstiles_block


In [174]:
turnstiles_block['delta_entries'] = turnstiles_block.apply(get_counts_entry, axis=1, max_counter=1e5)

29 139012.0
148 1494569141.0
2214 14129759.0
335599567 1374824.0
186916225 604712476.0
329 11078609.0
1794 239660.0
1820 9170515.0
76 564040.0


In [175]:
turnstiles_block['delta_exits'] = turnstiles_block.apply(get_counts_exit, axis=1, max_counter=1e5)
turnstiles_block

2 108703.0
19 122365.0
54 135367424.0
58 12723414.0
201390933 803021.0
2 6881370.0
835479154 869165449.0
247 6942597.0
46 102417.0
24 7344161.0
1458032 1326851.0
1327032 1458032.0
164 1021615.0


,c_a,unit,scp,station,date_time,exits,entries,prev_datetime,prev_entries,prev_exits,delta_entries,delta_exits
1,A002,R051,02-00-00,59 ST,2016-08-27 04:00:00,1966044,5799463,2016-08-27 00:00:00,5799442.0,1966041.0,21.0,3.0
2,A002,R051,02-00-00,59 ST,2016-08-27 08:00:00,1966079,5799492,2016-08-27 04:00:00,5799463.0,1966044.0,29.0,35.0
3,A002,R051,02-00-00,59 ST,2016-08-27 12:00:00,1966155,5799610,2016-08-27 08:00:00,5799492.0,1966079.0,118.0,76.0
4,A002,R051,02-00-00,59 ST,2016-08-27 16:00:00,1966214,5799833,2016-08-27 12:00:00,5799610.0,1966155.0,223.0,59.0
5,A002,R051,02-00-00,59 ST,2016-08-27 20:00:00,1966271,5800121,2016-08-27 16:00:00,5799833.0,1966214.0,288.0,57.0
...,...,...,...,...,...,...,...,...,...,...,...,...
177192,TRAM2,R469,00-05-01,RIT-ROOSEVELT,2016-09-02 05:00:00,259,5554,2016-09-02 01:00:00,5554.0,259.0,0.0,0.0
177193,TRAM2,R469,00-05-01,RIT-ROOSEVELT,2016-09-02 09:00:00,259,5554,2016-09-02 05:00:00,5554.0,259.0,0.0,0.0
177194,TRAM2,R469,00-05-01,RIT-ROOSEVELT,2016-09-02 13:00:00,259,5554,2016-09-02 09:00:00,5554.0,259.0,0.0,0.0
177195,TRAM2,R469,00-05-01,RIT-ROOSEVELT,2016-09-02 17:00:00,259,5554,2016-09-02 13:00:00,5554.0,259.0,0.0,0.0


In [176]:
# mf = turnstiles_block.groupby(['unit', 'station', 'date_time',]).agg({'delta_exits': 'sum'}).reset_index()
# mf[:50]

In [177]:



# top10 = (mf.sort_values(by='delta_exits',ascending=False).station.head(10))
# #pd.DataFrame(top10)
# top10

In [178]:
stations_hourly = (turnstiles_block.groupby(['station','date_time'])['delta_exits'].sum().reset_index())

In [179]:
stations_hourly[:50]

,station,date_time,delta_exits
0,1 AV,2016-08-27 04:00:00,1020.0
1,1 AV,2016-08-27 08:00:00,969.0
2,1 AV,2016-08-27 12:00:00,1322.0
3,1 AV,2016-08-27 16:00:00,4737.0
4,1 AV,2016-08-27 20:00:00,4519.0
5,1 AV,2016-08-28 00:00:00,3907.0
6,1 AV,2016-08-28 04:00:00,1214.0
7,1 AV,2016-08-28 08:00:00,606.0
8,1 AV,2016-08-28 12:00:00,2040.0
9,1 AV,2016-08-28 16:00:00,3780.0


In [180]:
top = stations_hourly.groupby(['station', 'date_time'])['delta_exits'].sum().reset_index()

        

top10 = (top.sort_values(by=['delta_exits'],ascending=False)
        .station.head(60))

In [181]:
top10.unique()

array(['34 ST-PENN STA', '34 ST-HERALD SQ', '47-50 STS ROCK',
       'TIMES SQ-42 ST', '14 ST-UNION SQ', 'GRD CNTRL-42 ST',
       'LEXINGTON AV/53', 'CHAMBERS ST', '59 ST', '86 ST',
       '59 ST COLUMBUS'], dtype=object)

In [182]:
top10.head(20)

2823      34 ST-PENN STA
2835      34 ST-PENN STA
2847      34 ST-PENN STA
2811      34 ST-PENN STA
2723     34 ST-HERALD SQ
2717     34 ST-HERALD SQ
2729     34 ST-HERALD SQ
3341      47-50 STS ROCK
3335      47-50 STS ROCK
3329      47-50 STS ROCK
14897     TIMES SQ-42 ST
14909     TIMES SQ-42 ST
2735     34 ST-HERALD SQ
14903     TIMES SQ-42 ST
2711     34 ST-HERALD SQ
2857      34 ST-PENN STA
14915     TIMES SQ-42 ST
2731     34 ST-HERALD SQ
2859      34 ST-PENN STA
2719     34 ST-HERALD SQ
Name: station, dtype: object

In [183]:
top.iloc[2823]

station             34 ST-PENN STA
date_time      2016-08-30 20:00:00
delta_exits                  42007
Name: 2823, dtype: object

In [184]:
mask = (stations_hourly['station'] == '34 ST-PENN STA')
stations_hourly[mask].head(50)

,station,date_time,delta_exits
2779,34 ST-PENN STA,2016-08-27 04:00:00,1532.0
2780,34 ST-PENN STA,2016-08-27 06:00:00,426.0
2781,34 ST-PENN STA,2016-08-27 08:00:00,3144.0
2782,34 ST-PENN STA,2016-08-27 10:00:00,3560.0
2783,34 ST-PENN STA,2016-08-27 12:00:00,11221.0
2784,34 ST-PENN STA,2016-08-27 14:00:00,5630.0
2785,34 ST-PENN STA,2016-08-27 16:00:00,12498.0
2786,34 ST-PENN STA,2016-08-27 18:00:00,5896.0
2787,34 ST-PENN STA,2016-08-27 20:00:00,13533.0
2788,34 ST-PENN STA,2016-08-27 22:00:00,3952.0
